In [1]:
import json
import csv

# Load the dependency JSON file
with open("dependencies.json", "r") as f:
    data = json.load(f)

# Initialize dictionaries for Fan-In and Fan-Out
fan_in = {}
fan_out = {}

# Iterate over each module in the JSON data
for module, details in data.items():
    if isinstance(details, dict):
        # Fan-Out is the number of modules this module imports.
        imports = details.get("imports", [])
        fan_out[module] = len(imports)
        
        # Fan-In is the number of modules that import this module.
        imported_by = details.get("imported_by", [])
        fan_in[module] = len(imported_by)
    else:
        # If details is not a dictionary, skip it.
        continue

# Write the results to a CSV file
with open("fan_in_out.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Module", "Fan-In", "Fan-Out"])
    # Sorting modules alphabetically for readability
    for module in sorted(data.keys()):
        # Use 0 if a key is missing (should not happen with your structure)
        writer.writerow([module, fan_in.get(module, 0), fan_out.get(module, 0)])

print("✅ Fan-In and Fan-Out data has been saved to fan_in_out.csv.")


✅ Fan-In and Fan-Out data has been saved to fan_in_out.csv.


In [2]:
import pandas as pd
df = pd.read_csv("fan_in_out.csv")

In [3]:
df

,Module,Fan-In,Fan-Out
0,__main__,0,78
1,charset_normalizer,2,0
2,charset_normalizer.constant,1,0
3,httpie,36,0
4,httpie.__main__,2,3
...,...,...,...
108,rich.text,6,3
109,rich.theme,2,1
110,urllib3,7,2
111,urllib3.util,6,1


In [4]:
import json
import csv
import networkx as nx

# Load the dependency JSON file
with open("dependencies.json", "r") as f:
    data = json.load(f)

# Create a directed graph based on "imports"
G = nx.DiGraph()
for module, details in data.items():
    if isinstance(details, dict):
        imports = details.get("imports", [])
        for imp in imports:
            G.add_edge(module, imp)

# Detect cycles using networkx
cycles = list(nx.simple_cycles(G))

# Save the cycles to a CSV file
with open("cyclic_dependencies.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Cycle Number", "Cycle Path"])
    for idx, cycle in enumerate(cycles, start=1):
        writer.writerow([idx, " -> ".join(cycle)])

if cycles:
    print("🔴 Cyclic dependencies found and saved to cyclic_dependencies.csv.")
else:
    print("✅ No cyclic dependencies detected.")


🔴 Cyclic dependencies found and saved to cyclic_dependencies.csv.


In [5]:
import pandas as pd
df2 = pd.read_csv("cyclic_dependencies.csv")
df2

,Cycle Number,Cycle Path
0,1,httpie.manager.tasks
1,2,requests
2,3,rich
3,4,urllib3
4,5,pygments
5,6,pygments.formatters
6,7,pygments.lexers
7,8,pygments.styles
8,9,rich.table
9,10,httpie.__main__ -> httpie.core -> httpie.inter...


In [6]:
import json
import csv

# Load the dependency JSON file
with open("dependencies.json", "r") as f:
    data = json.load(f)

# Gather modules that are keys and those that appear in any "imports"
modules_in_keys = set(data.keys())
modules_in_imports = set()
unused = []  # Modules defined in the JSON that have no connections

for module, details in data.items():
    if isinstance(details, dict):
        imports = details.get("imports", [])
        imported_by = details.get("imported_by", [])
        modules_in_imports.update(imports)
        # Consider a module "unused/disconnected" if both lists are empty
        if not imports and not imported_by:
            unused.append(module)

# Modules that appear in "imports" but are not defined as keys (external or not captured)
external_modules = modules_in_imports - modules_in_keys

with open("unused_modules.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Unused/Disconnected Modules (Defined in JSON)"])
    for module in sorted(unused):
        writer.writerow([module])
    writer.writerow([])
    writer.writerow(["External Modules (Appearing in Imports but not defined)"])
    for module in sorted(external_modules):
        writer.writerow([module])

print("✅ Unused/disconnected modules saved to unused_modules.csv.")


✅ Unused/disconnected modules saved to unused_modules.csv.


In [8]:
df3 = pd.read_csv("unused_modules.csv")

In [9]:
df3

,Unused/Disconnected Modules (Defined in JSON)
0,External Modules (Appearing in Imports but not...


In [10]:
import json
import csv

# Load dependency JSON
with open("dependencies.json", "r") as f:
    data = json.load(f)

def max_depth(module, visited=None):
    if visited is None:
        visited = set()
    # Avoid cycles: if module already visited, stop recursion
    if module in visited:
        return 0
    visited.add(module)
    details = data.get(module, {})
    # Get the list of imported modules (if any)
    imports = details.get("imports", []) if isinstance(details, dict) else []
    if not imports:
        return 0
    # For each imported module, calculate depth recursively (using a copy of visited)
    return max((1 + max_depth(imp, visited.copy())) for imp in imports)

# Compute depth for each module (defined in our JSON)
depths = {}
for module in data.keys():
    depths[module] = max_depth(module)

# Save dependency depths to a CSV file
with open("dependency_depth.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Module", "Dependency Depth"])
    for module, depth in sorted(depths.items(), key=lambda x: x[1], reverse=True):
        writer.writerow([module, depth])

print("✅ Dependency depth data has been saved to dependency_depth.csv.")


✅ Dependency depth data has been saved to dependency_depth.csv.


In [11]:
df4 = pd.read_csv("dependency_depth.csv")
df4

,Module,Dependency Depth
0,__main__,25
1,httpie.manager.__main__,24
2,httpie.manager.core,23
3,httpie.manager.tasks,22
4,httpie.internal.daemon_runner,21
...,...,...
108,pygments.util,0
109,requests.__version__,0
110,requests.cookies,0
111,rich.filesize,0


In [12]:
import pandas as pd

In [13]:
df5  = pd.read_csv("TypeMetrics.csv")
df5

,Project Name,Package Name,Type Name,LCOM1,LCOM2,LCOM3,LCOM4,LCOM5,YALCOM
0,.,com.iluwatar.saga.orchestration,ChapterResult,5.0,4.0,3.0,3.0,0.833333,0.75
1,.,com.iluwatar.saga.orchestration,FlyBookingService,0.0,0.0,1.0,1.0,0.000000,0.00
2,.,com.iluwatar.saga.orchestration,HotelBookingService,1.0,0.0,2.0,1.0,0.000000,0.00
3,.,com.iluwatar.saga.orchestration,OrchestrationChapter,3.0,0.0,3.0,3.0,0.000000,-1.00
4,.,com.iluwatar.saga.orchestration,OrderService,0.0,0.0,1.0,1.0,0.000000,0.00
5,.,com.iluwatar.saga.orchestration,Saga,4.0,0.0,2.0,2.0,0.625000,0.40
6,.,com.iluwatar.saga.orchestration,Chapter,0.0,0.0,1.0,1.0,0.000000,-1.00
7,.,com.iluwatar.saga.orchestration,SagaApplication,3.0,0.0,3.0,1.0,0.000000,0.00
8,.,com.iluwatar.saga.orchestration,SagaOrchestrator,16.0,0.0,1.0,1.0,0.725000,0.00
9,.,com.iluwatar.saga.orchestration,CurrentState,6.0,0.0,1.0,1.0,0.416667,0.00


In [16]:
import pandas as pd

# Load the LCOM results CSV file
df = pd.read_csv("TypeMetrics.csv")

# Create a new column 'MaxLCOM' that holds the maximum value among LCOM1 to LCOM5 for each class
df['MaxLCOM'] = df[['LCOM1', 'LCOM2', 'LCOM3', 'LCOM4', 'LCOM5']].max(axis=1)

# Define a threshold for high LCOM values (subjective, e.g., > 5.0)
threshold = 5.0

# Filter classes where MaxLCOM exceeds the threshold
high_lcom = df[df['MaxLCOM'] >= threshold]
high_lcom
# Display the filtered classes with their LCOM values and the computed MaxLCOM
print("=== Classes with High LCOM Values ===")
print(high_lcom[['Package Name', 'Type Name', 'LCOM1', 'LCOM2', 'LCOM3', 'LCOM4', 'LCOM5', 'YALCOM', 'MaxLCOM']])


=== Classes with High LCOM Values ===
                      Package Name         Type Name  LCOM1  LCOM2  LCOM3  \
0  com.iluwatar.saga.orchestration     ChapterResult    5.0    4.0    3.0   
8  com.iluwatar.saga.orchestration  SagaOrchestrator   16.0    0.0    1.0   
9  com.iluwatar.saga.orchestration      CurrentState    6.0    0.0    1.0   

   LCOM4     LCOM5  YALCOM  MaxLCOM  
0    3.0  0.833333    0.75      5.0  
8    1.0  0.725000    0.00     16.0  
9    1.0  0.416667    0.00      6.0  


In [17]:
high_lcom

,Project Name,Package Name,Type Name,LCOM1,LCOM2,LCOM3,LCOM4,LCOM5,YALCOM,MaxLCOM
0,.,com.iluwatar.saga.orchestration,ChapterResult,5.0,4.0,3.0,3.0,0.833333,0.75,5.0
8,.,com.iluwatar.saga.orchestration,SagaOrchestrator,16.0,0.0,1.0,1.0,0.725000,0.00,16.0
9,.,com.iluwatar.saga.orchestration,CurrentState,6.0,0.0,1.0,1.0,0.416667,0.00,6.0
